# Disseration Experiment 5h
# Generate ANCHOR Output (Credit Default) January Eight¶
Ciaran Finnegan January 2023

# Import Libraries + Custom Functions

## Import Libraries

In [ ]:
# Import libs
import numpy as np
import pandas as pd

# Display libraries
from IPython.display import display, HTML
from prettytable import PrettyTable

# Import necessary libraries for ANN model building
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Import necessary library for ANCHOR explainer
import alibi
from alibi.explainers import AnchorTabular
#import anchor
from anchor import anchor_tabular

# Libraries required for metrics calculations
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import warnings

# Compute additional evaluation metrics
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from sklearn.metrics import precision_score, recall_score, f1_score

# Classifier training (not used for explainability)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Additional display libraires
import contextlib
import sys
from contextlib import contextmanager

# Libraries used in Experiment Creation of XL Output Metrics
import os
import time
import random
import openpyxl

## Custom Functions

Dataset Visualisations

In [ ]:
%run ./DS_Visualisation_Functions.ipynb

Metrics

In [ ]:
%run ./XAI_Metrics_Functions.ipynb

Model Evaluation Functions

In [ ]:
%run ./DS_Model_Build_Evaluation_Functions.ipynb

Track Experiment Result Functions

In [ ]:
%run ./XAI_Experiment_Functions.ipynb

------------------------------------

In [ ]:
ds_file_to_load = 'credit_default_data.csv'
df = pd.read_csv(ds_file_to_load)

In [ ]:
# Determine the threshold for missing values
threshold = 0.75 * len(df)

# Identify columns with missing values greater than the threshold
missing_columns = df.columns[df.isnull().sum() > threshold]

# Print the columns with more than 75% missing values
print("Columns with more than 75% missing values:", missing_columns)

# Drop columns with missing values greater than the threshold
df = df.drop(columns=missing_columns)

# Save or continue processing with columns removed that had high volumes of missing data


## Categorical Data 

In [ ]:
# List of categorical columns
cat_cols = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

In [ ]:
# One-hot encode categorical variables
df_encoded = pd.get_dummies(df, columns=cat_cols)

In [ ]:
# Check the distribution of the target variable
target_distribution = df_encoded['default'].value_counts()

target_distribution

In [ ]:
# Separate the majority and minority classes
df_majority = df_encoded[df_encoded['default'] == 0]
df_minority = df_encoded[df_encoded['default'] == 1]

In [ ]:
# Downsample the majority class
df_majority_downsampled = resample(df_majority, 
                                   replace=False, 
                                   n_samples=target_distribution[1], 
                                   random_state=42)

In [ ]:
# Combine the downsampled majority class with the minority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

In [ ]:
# Shuffle the dataset to mix the data points
df_downsampled = df_downsampled.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# Display the distribution of the target variable in the downsampled dataset
df_downsampled['default'].value_counts()

In [ ]:
# Splitting the features and target variable
X = df_downsampled.drop('default', axis=1)
y = df_downsampled['default']

In [ ]:
# Splitting the data into training and testing sets
X_train_downsampled, X_test_downsampled, y_train_downsampled, y_test_downsampled = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
# Reset Indexes
X_train_downsampled = X_train_downsampled.reset_index(drop=True)
X_test_downsampled = X_test_downsampled.reset_index(drop=True)

y_train_downsampled = y_train_downsampled.reset_index(drop=True)
y_test_downsampled = y_test_downsampled.reset_index(drop=True)

In [ ]:
X_train_downsampled, X_test_downsampled = scale_the_features(X_train_downsampled, X_test_downsampled, df_downsampled)

# Build Model

## Set Up Hyperparameters

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [ ]:
# Building the model
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_downsampled.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

## Build Neural Network (w/TensorFlow/Keras)

In [ ]:
# Training the model
history = model.fit(X_train_downsampled, y_train_downsampled, epochs=15, batch_size=32, validation_split=0.2, verbose=1)
print("Model trained successfully!")

# Load Model

A Neural Network Model has been created in another Kubeflow Notebook and is being used in all the XAI experiments

In [ ]:
loaded_model = keras.models.load_model('ccfraud_model')  # If saved as SavedModel

In [ ]:
X_test_loaded, y_test_loaded, X_train_loaded, y_train_loaded, df_downsampled_loaded, dfCatCols = load_CC_train_test_data()

In [ ]:
X_train_loaded.head(1)

In [ ]:
y_train_loaded.head(2)

## Re-Display Model Peformance

For illustration, the evualtion metrics of the NN model will be repeated here.

### Tabular Data

In [ ]:
scaler = StandardScaler()   
X_test_loaded_scaled = scaler.fit_transform(X_test_loaded)

In [ ]:
y_pred_loaded = display_model_metrics_tabular(loaded_model, X_test_loaded_scaled, y_test_loaded)

### Confusion Matrix

In [ ]:
generate_confusion_matrix(y_test_loaded, y_pred_loaded)

# Generate ANCHOR Values

#### Suppress Warnings to clean up output

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

Check layout of X_train_downsampled

In [ ]:
X_train_downsampled

In [ ]:
# Extract the feature names, excluding the target variable 'default'
column_names = df_downsampled.drop('default', axis=1).columns

In [ ]:
# Convert NumPy array to DataFrame
X_train_downsampled = pd.DataFrame(X_train_downsampled, columns=column_names)

In [ ]:
X_train_downsampled.head()

In [ ]:
#from anchor import anchor_tabular

In [ ]:
# Separate the features and the target variable
X = df_encoded.drop('default', axis=1)
y = df_encoded['default']

In [ ]:
# Define the explainer
explainer = anchor_tabular.AnchorTabularExplainer(
    class_names=['Not Default', 'Default'],
    feature_names=X.columns.tolist(),
    train_data=X_train_downsampled.values,
    categorical_names={}
)

In [ ]:
import contextlib
import numpy as np
import os
import sys
from contextlib import contextmanager

In [ ]:
@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as fnull:
        with contextlib.redirect_stdout(fnull), contextlib.redirect_stderr(fnull):
            yield None

In [ ]:
def predict_fn(x):
    # Ensure x is in batch format
    if len(x.shape) == 1:
        x = np.expand_dims(x, axis=0)
    # Suppress the output of the progress bar
    with suppress_stdout():
        # Get the model's prediction (probability of the positive class)
        probabilities = model.predict(x, verbose=0)
    # Convert probabilities to class labels (0 or 1)
    labels = (probabilities > 0.5).astype(int)
    return labels.flatten()

In [ ]:
# Convert NumPy array to DataFrame
X_test_downsampled = pd.DataFrame(X_test_downsampled, columns=column_names)

In [ ]:
# Ensure that the instance passed to explain_instance is in the correct shape
idx = 0
instance_to_explain = X_test_downsampled.iloc[idx].values.reshape(1, -1)

In [ ]:
# Generate an explanation for the first instance in the test set
exp = explainer.explain_instance(instance_to_explain, predict_fn, threshold=0.95)

In [ ]:
# Show the explanation
exp.show_in_notebook()

In [ ]:
exp.show_in_notebook(show_table=True, show_all=False)

#### Pseudocode to Generate Initial ANCHOR Values

For the RF model built above in Python, select a random sample 
of 15 instances in the test data, 10 for Class '0' and 5 for 
Class '1', and generate ANCHOR values as explainers for these  
instances in the test dataset.

Present these ANCHOR values in an easily understood and pleasant 
on the eye tabular output format for the Python Kubeflow Notebook
in which I am writing my Python code. 

Create a second tabular format what shows an equally appealing 
output in my Python Notebook that shows the ANCHOR values and the
feature details for each instance on a single row, across which I
can scroll.

Comment each line of Python code with as much detail as practical. 

Output the ANCHOR values to a CSV file. Output the feature details 
for each corresponding instance for which the ANCHOR Values were
created in a seperate CSV file.

After the code generation provide as much narrative detail 
as possible.

##### Further pseudocode...

Use the AnchorTabular explainer from the alibi library. This explainer provides local explanations for classification models' predictions by identifying a minimal set of conditions (features) in the instance that ensure the model's decision remains unchanged (these conditions are called "anchors").

The steps:

Select a random sample of 15 instances from the test data, 10 from Class '0' and 5 from Class '1'.
Set up the AnchorTabular explainer and fit it to the training data.
Generate anchor explanations for the selected instances.
Present the anchor values in two tabular formats: a summary table and a detailed table.
Output the anchor values and feature details to CSV files.

In [ ]:
# Loop through the first five instances in the test dataset
for idx in range(5):
    instance = X_test_downsampled.iloc[idx].values.reshape(1, -1)
    print(f"\nInstance {idx + 1}:")
    
    # Generate an explanation for the instance
    exp = explainer.explain_instance(instance, predict_fn, threshold=0.95)
    
    # Show the explanation in the notebook
    exp.show_in_notebook()

### Create an ANCHOR File Output

In [ ]:
# Initialize a list to store the ANCHOR results
anchor_results = []

In [ ]:
import re

anchor_results = []

# Loop through the first five instances in the test dataset
for idx in range(5):
    instance = X_test_downsampled.iloc[idx].values.reshape(1, -1)
    # Generate an explanation for the instance
    exp = explainer.explain_instance(instance, predict_fn, threshold=0.95)
    
    # Extract feature importance from the explanation
    feature_importance = {}
    for condition in exp.names():
        # Handle conditions with '='
        if '=' in condition:
            feature, value = condition.split('=')
            feature = feature.strip()
            value = float(value.strip())
            feature_importance[feature] = ('=', value)
        # Handle conditions with '>' or '<'
        elif '>' in condition or '<' in condition:
            parts = re.split('([><])', condition)
            feature, operator, value = [part.strip() for part in parts if part.strip()]
            value = float(value)
            feature_importance[feature] = (operator, value)
        else:
            raise ValueError(f"Unexpected format for ANCHOR explanation: {condition}")

    anchor_results.append(feature_importance)

In [ ]:
# Create a DataFrame from the results
df_anchor_results = pd.DataFrame(anchor_results)

In [ ]:
# Show the DataFrame
print(df_anchor_results)

In [ ]:
# Write the DataFrame to a CSV file
df_anchor_results.to_csv('anchor_results_ANN.csv', index=False)

-----

# Prepare ANCHOR Values for Metrics

In [ ]:
@timeit
def generate_anchors_for_instances(df, num_instances=5):
    # Initialize a list to store the ANCHOR results
    new_anchor_results = []
    feature_instances = []

    # Loop through the first five instances in the test dataset
    for idx in range(num_instances):
        #instance = X_test_downsampled.iloc[idx]
        instance = df.iloc[idx]
        feature_instances.append(instance)

        # Generate an explanation for the instance with a lower threshold
        exp = explainer.explain_instance(instance.values.reshape(1, -1), predict_fn, threshold=0.99)

        # Check if an explanation was found
        if exp is not None:
            # Parse the conditions from the explanation and format them
            anchor_explanation = []
            for condition in exp.names():
                if ' > ' in condition or ' < ' in condition:
                    feature, relation, value = condition.split(' ')[0], condition.split(' ')[1], condition.split(' ')[2]
                    try:
                        anchor_explanation.append(f"'{feature} {relation} {float(value):.2f}'")
                    except ValueError:
                        anchor_explanation.append(f"'{condition}'")
                else:
                    anchor_explanation.append(f"'{condition}'")

            # Convert the list of strings to a single string
            anchor_explanation_str = '[' + ', '.join(anchor_explanation) + ']'

            # Add the formatted explanation to the results list
            new_anchor_results.append(anchor_explanation_str)
        else:
            new_anchor_results.append("['No explanation found']")

    # Create a DataFrame from the results
    new_df_anchor_results = pd.DataFrame(new_anchor_results, columns=['Anchor Explanation'])

    # Create a DataFrame from the feature instances
    df_feature_instances = pd.DataFrame(feature_instances)
    
    
    return df_feature_instances, new_df_anchor_results

In [ ]:
#df_feature_instances, new_df_anchor_results = generate_anchors_for_instances(X_test_downsampled, 5)
results, exec_time = generate_anchors_for_instances(X_test_downsampled, 5)

In [ ]:
df_feature_instances, new_df_anchor_results = results

## Determine Computational Efficiency Value

In [ ]:
# Display time to generate DiCE explainers
print(f"ANCHORS Execution Time: {exec_time} seconds")

In [ ]:
# Show the DataFrames
print("Anchor Explanations:")
print(new_df_anchor_results)
#print("\nFeature Instances:")
#print(df_feature_instances)

# Write the DataFrames to CSV files
new_df_anchor_results.to_csv('new_anchor_results5.csv', index=False)
df_feature_instances.to_csv('feature_instances5.csv', index=False)

In [ ]:
df_feature_instances.head()

In [ ]:
new_df_anchor_results

## Parse the Anchor Explanations

In [ ]:
import ast

In [ ]:

# Step 2: Convert the 'Anchor Explanation' column from a string representation of a list back to an actual list
new_df_anchor_results['Anchor Explanation'] = new_df_anchor_results['Anchor Explanation'].apply(ast.literal_eval)

# Step 3: Determine the maximum number of conditions in the ANCHOR explanations across all instances
max_num_conditions = max(new_df_anchor_results['Anchor Explanation'].apply(len))

# Step 4: Initialize a list to store the numerical representations of the ANCHOR explanations
numerical_explanations = []

# Step 5: Loop through each ANCHOR explanation and convert it to a numerical representation
for explanation in new_df_anchor_results['Anchor Explanation']:
    numerical_representation = [-1] * len(df_feature_instances.columns) * max_num_conditions
    for idx, condition in enumerate(explanation):
        # Parse the condition to extract the feature name and value
        feature, relation, value = condition.split(' ')[0], condition.split(' ')[1], condition.split(' ')[2]
        
        # Find the index of the feature in the feature dataframe
        feature_idx = df_feature_instances.columns.get_loc(feature)
        
        # Store the feature index in the numerical representation
        numerical_representation[feature_idx * max_num_conditions + idx] = float(value)
    numerical_explanations.append(numerical_representation)

# Step 6: Create a dataframe from the numerical representations
df_anchors_numerical = pd.DataFrame(numerical_explanations)

# Display the resulting dataframe
print(df_anchors_numerical)


In [ ]:
def parse_anchor_exps(new_df_anchor_results):
    
    # Step 1: Convert the 'Anchor Explanation' column from a string representation of a list back to an actual list
    new_df_anchor_results['Anchor Explanation'] = new_df_anchor_results['Anchor Explanation'].apply(ast.literal_eval)

    # Step 2: Determine the maximum number of conditions in the ANCHOR explanations across all instances
    max_num_conditions = max(new_df_anchor_results['Anchor Explanation'].apply(len))

    # Step 3: Initialize a list to store the numerical representations of the ANCHOR explanations
    numerical_explanations = []

    # Step 4: Loop through each ANCHOR explanation and convert it to a numerical representation
    for explanation in new_df_anchor_results['Anchor Explanation']:
        numerical_representation = [-1] * len(df_feature_instances.columns) * max_num_conditions
        for idx, condition in enumerate(explanation):
            # Parse the condition to extract the feature name and value
            feature, relation, value = condition.split(' ')[0], condition.split(' ')[1], condition.split(' ')[2]

            # Find the index of the feature in the feature dataframe
            feature_idx = df_feature_instances.columns.get_loc(feature)

            # Store the feature index in the numerical representation
            numerical_representation[feature_idx * max_num_conditions + idx] = float(value)
        numerical_explanations.append(numerical_representation)

    # Step 5: Create a dataframe from the numerical representations
    df_anchors_numerical = pd.DataFrame(numerical_explanations)

    # Display the resulting dataframe
    print(df_anchors_numerical)
    
    return df_anchors_numerical

In [ ]:
new_df_anchor_results

## Display ANCHORS

In [ ]:
# Display the first few rows of each dataset to understand their structure
instance_features_head = df_feature_instances.head()
anchor_explanations_head = new_df_anchor_results.head()
anchor_explanations_numerical = df_anchors_numerical.head()

In [ ]:
instance_features_head

In [ ]:
anchor_explanations_head

In [ ]:
anchor_explanations_numerical

## Generate Outfile for review

In [ ]:
anchors_num_explainers_filepath = "anchor_numerical_explainers.csv"

anchor_explanations_numerical.to_csv(anchors_num_explainers_filepath, index=False)

# Generate XAI Metrics 

## Identity Metric 

#### Run a Basic Test First

In [ ]:
# Select two random instances from the ANCHOR dataframe
df_xai_numerical = anchor_explanations_numerical

random_indices = np.random.choice(df_xai_numerical.index, size=2, replace=False)
instance_1 = df_xai_numerical.iloc[random_indices[0]]
instance_2 = df_xai_numerical.iloc[random_indices[1]]

# Compute the Euclidean distance between the selected instances - uses custom project function
distance = get_euclidean_distance(instance_1, instance_2)
print(f"Euclidean distance between instance {random_indices[0]} and instance {random_indices[1]}: {distance:.4f}")

#### Retrieve Identity Score

In [ ]:
from scipy.spatial import distance

In [ ]:
ANCHOR_Identity_Metric = get_identity_metric(df_feature_instances, anchor_explanations_numerical, "ANCHOR")

#### Display Identity Score Metric

In [ ]:
ANCHOR_Identity_Number = "{:.2f}%".format(ANCHOR_Identity_Metric)
display_text("ANCHOR Identity Metric Score: " + ANCHOR_Identity_Number)

## Stability Metric

### Invoke Stability Metric Function

In [ ]:
ANCHOR_Stability_Metric = get_stability_metric_y(anchor_explanations_numerical, y_test_downsampled, 'ANCHOR')

#### Display Stability Score Metric

In [ ]:
ANCHOR_Stability_Number = "{:.2f}%".format(ANCHOR_Stability_Metric)
display_text("ANCHOR Stability Metric Score: " + ANCHOR_Stability_Number)

## Seperability

### Invoke Seperability Metric Function

#### Retrieve Seperability Score

In [ ]:
ANCHOR_Seperability_Metric = get_seperability_metric(df_feature_instances, anchor_explanations_numerical, "ANCHOR")

#### Display Seperability Score Metric

In [ ]:
ANCHOR_Seperability_Number = "{:.2f}%".format(ANCHOR_Seperability_Metric)
display_text("ANCHOR Seperability Metric Score: " + ANCHOR_Seperability_Number)

## Similarity

### Invoke Similarity Metric Function

#### Retrieve Similarity Score

In [ ]:
ANCHOR_Similarity_Metric = get_similarity_metric(df_feature_instances, anchor_explanations_numerical, "ANCHOR", 
                                                 use_dbscan=False)

#### Display Similarity Score Metric

In [ ]:
ANCHOR_Similarity_Number = "{:6.2f}".format(ANCHOR_Similarity_Metric)
display_text("ANCHOR Similarity Metric Value: " + ANCHOR_Similarity_Number)

----------------------------------------

# XAI Experiments - Metrics Capture

## Suppress Warnings to clean up output

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

## Break out Model Test Data into a list of dataframes

### Create Test Data for Experiment Input

In [ ]:
original_feature_names = [col for col in df_downsampled_loaded.columns if col != 'Fraud']

# Ensure X_test_loaded has the correct column names (if necessary)
X_test_loaded.columns = original_feature_names

In [ ]:
# Combine X_test_loaded and y_test into a single DataFrame
df_TestData = pd.concat([X_test_loaded, y_test_loaded], axis=1)

### Split the DataFrame into 20 consecutive smaller DataFrames

In [ ]:
# Split the DataFrame into 20 consecutive smaller DataFrames
split_size, list_df = split_TestData_into_nn_Blocks(df_TestData, num_splits = 20)

In [ ]:
# Count the occurrence of each unique value in the 'Fraud' column
fraud_counts = df_TestData['Fraud'].value_counts()

# Display the counts
print("Breakdown of 'Fraud' and non-Fraud label records in df_TestData:")
print(fraud_counts)

### Add a routine to check output values

In [ ]:
# Display starting points in the first nn sub dataframes
startBlockDisplay(df_TestData, split_size, 1)

## Confirm Starting Point in External ANCHORS XAI XL File

The code below acts so that for each dataframe in the list just created the following actions are carried out;

Check if an XAI results XL spreadsheet called 'ANCHOR_XAI_Metrics_Experiments.xls' exists;

If not create an empty XL spreadsheet with the name 'ANCHOR_XAI_Metrics_Experiments.xls', and then define a variable called ‘Sample’ with an integer value of 1 and print the value of 'Sample' to output.

If and XL spreadsheet called 'ANCHOR_XAI_Metrics_Experiments.xls' does exist, then read the entries in the spreadsheet in the first column named ‘Sample Number’ and create a variable in this Python program named ‘Sample’ that is one integer value higher than the highest integer number column named ‘Sample Number’ in the XL, and print this value of 'Sample' to output.

In [ ]:
# Create a sequential number as an identifier for each DataFrame
list_df = {f'df_{i + 1}': list_df[i] for i in range(len(list_df))}

In [ ]:
# File path for the ANCHOR XAI metrics results spreadsheet
ANCHOR_xai_file_path = 'ANCHOR_XAI_Metrics_Experiments.xlsx'  # Stored locally

In [ ]:
# Call Function to update or create the spreadsheet and determine the 'Sample' number
# Process each dataframe in 'list_df'
sample = return_next_sample_number_to_process(list_df, LIME_xai_file_path, "LIME")

## Select Next Dataframe to Process

---------------------------------

	
Extend the Python code so that the code reads in the dataframe from 'list df' that corresponds to the integer value in the 
variable named ‘Sample’. 

Assign this dataframe the name 'df_Selected_from_List'.


----------------------------------

### Initialize Dataframe to Capture Re-start Point as None

In [ ]:
# Initialize df_Selected_from_List as None
df_Selected_from_List = None

### Extract test data block to restart XAI metrics process

In [ ]:
df_Selected_from_List, key = select_restart_testdata_block(df_Selected_from_List, 
                                                           list_df, 
                                                           LIME_xai_file_path)

In [ ]:
# If no DataFrame is selected (e.g., if 'Sample' exceeds the number of DataFrames in list_df)
if 'df_Selected_from_List' not in locals():
    print("No DataFrame selected. The 'Sample' number may exceed the number of DataFrames in list_df.")

-----------------------------

## Generate XAI Metrics from Dataframe

### Generate the ANCHOR Values for the Test Data Block

In [ ]:
df_Selected_from_List.head(2)

In [ ]:
print(df_Selected_from_List.index)

In [ ]:
df_Selected_from_List = df_Selected_from_List.drop('Fraud', axis=1)

In [ ]:
df_Selected_from_List.head(2)

In [ ]:
print(df_Selected_from_List.index)

In [ ]:
# Dynamically determine start and end indices
start_index = df_Selected_from_List.index[0]  # First index of the DataFrame
print(f'Start Index is : {start_index}')

In [ ]:
end_index = df_Selected_from_List.index[-1]  # Last index of the DataFrame
print(f'End Index is : {end_index}')

In [ ]:
index_range = list(range(start_index, end_index + 1))

In [ ]:
# Jan 6th - use new loaded model and data
default_1_indices = index_range

chosen_indices = list(np.random.choice(default_1_indices, 4)) 

In [ ]:
#results_LIME, exec_time_LIME = generate_lime_explanations(df_Selected_from_List, chosen_indices)
#results, exec_time = generate_anchors_for_instances(X_test_downsampled, 5)

results_ANCHOR, exec_time_ANCHOR = generate_anchors_for_instances(df_Selected_from_List)

In [ ]:
# Unpack the results to get features_df, lime_df
#features_df_LIME, lime_df = results_LIME
#df_feature_instances, new_df_anchor_results = results

df_feature_anchor_instances, new_df_anchor_results = results_ANCHOR

In [ ]:
df_anchors_numerical = parse_anchor_exps(new_df_anchor_results)

-----------------------------

### Generate Identity Metric

#### Run a Basic Test First

In [ ]:
# Select two random instances from the ANCHOR dataframe
df_xai_numerical = df_anchors_numerical

random_indices = np.random.choice(df_xai_numerical.index, size=2, replace=False)
instance_1 = df_xai_numerical.iloc[random_indices[0]]
instance_2 = df_xai_numerical.iloc[random_indices[1]]

# Compute the Euclidean distance between the selected instances - uses custom project function
distance = get_euclidean_distance(instance_1, instance_2)
print(f"Euclidean distance between instance {random_indices[0]} and instance {random_indices[1]}: {distance:.4f}")

#### Retrieve Identity Score

In [ ]:
from scipy.spatial import distance
ANCHOR_Identity_Metric = get_identity_metric(df_feature_anchor_instances, df_anchors_numerical, "ANCHOR")

#### Display Identity Score Metric

In [ ]:
ANCHOR_Identity_Number = "{:.2f}%".format(ANCHOR_Identity_Metric)
display_text("ANCHOR Identity Metric Score: " + ANCHOR_Identity_Number)

In [ ]:
# Read in XAI Metric for Identity
XAI_Ident_Metric_1 = ANCHOR_Identity_Metric

-------------------------

### Generate Stability Metric

#### Retrieve Stability Score

In [ ]:
ANCHOR_Stability_Metric = get_stability_metric_y(df_anchors_numerical, y_test_loaded, 'ANCHOR')

#### Display Stability Score Metric

In [ ]:
ANCHOR_Stability_Number = "{:.2f}%".format(ANCHOR_Stability_Metric)
display_text("ANCHOR Stability Metric Score: " + ANCHOR_Stability_Number)

In [ ]:
# Read in XAI Metric for Stability
XAI_Stability_Metric_2 = ANCHOR_Stability_Metric

----------------

### Generate Seperability Metric

#### Retrieve Seperability Score

In [ ]:
ANCHOR_Seperability_Metric = get_seperability_metric(df_feature_anchor_instances, df_anchors_numerical, "ANCHOR")

In [ ]:
Display Seperability Score Metric¶

In [ ]:
ANCHOR_Seperability_Number = "{:.2f}%".format(ANCHOR_Seperability_Metric)
display_text("ANCHOR Seperability Metric Score: " + ANCHOR_Seperability_Number)

In [ ]:
# Read in XAI Metric for Seperability
XAI_Seperability_Metric_3 = ANCHOR_Seperability_Metric

------------------------

### Generate Similarity Metric

#### Retrieve Similarity Score

In [ ]:
ANCHOR_Similarity_Metric = get_similarity_metric(df_feature_anchor_instances, df_anchors_numerical, "ANCHOR", 
                                                 use_dbscan=False)

#### Display Similarity Score Metric

In [ ]:
ANCHOR_Similarity_Number = "{:6.2f}".format(ANCHOR_Similarity_Metric)
display_text("ANCHOR Similarity Metric Value: " + ANCHOR_Similarity_Number)

In [ ]:
# Read in XAI Metric for Similarity
XAI_Similarity_Metric_4 = ANCHOR_Similarity_Metric

------------------------

### Display Final Set of Metrics (this run)

In [ ]:
# Print the results
print(f"XAI Ident Metric 1: {XAI_Ident_Metric_1}")
print(f"XAI Stability Metric 2: {XAI_Stability_Metric_2}")
print(f"XAI Seperability Metric 1: {XAI_Seperability_Metric_3}")
print(f"XAI Similarity Metric 1: {XAI_Similarity_Metric_4}")
print(f"XAI Time Metric 5: {exec_time_ANCHOR} seconds")

------------------------

## Write Out Metrics to XL

In [ ]:
write_xai_Metrics_to_XL(ANCHOR_xai_file_path, 
                        sample, 
                        ANCHOR_Identity_Metric, 
                        ANCHOR_Stability_Metric, 
                        ANCHOR_Seperability_Metric, 
                        ANCHOR_Similarity_Metric, 
                        exec_time_ANCHOR, 
                        df_Selected_from_List,
                        "ANCHOR")